In [1]:
%config IPCompleter.greedy=True

In [40]:
# !pip3 install quandl
# !python3 -m pip install --user backtesting
# !python3 -m pip install --user pyalgotrade
# !python3 -m pip install --user Zipline
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
from math import pi
import quandl as qdl
from scipy.stats import linregress
from getpass import getpass
import models.data_provider as dp

In [ ]:
dm = dp.DataModel("../xcels", ["master0.csv", "master1.csv"])
dm.read()

In [35]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG

df = dm.get(dm.TA_SYMBOLS[0])
df.columns = ['symbol', 'name', 'amount', 'volume', 'value', 'lastday', 'open',
       'close', 'last-change', 'last-percent', 'ending', 'ending-change',
       'ending-percent', 'min', 'max', 'year', 'month', 'day', 'diff_min_max',
       'diff_open', 'adj_scale', 'adj_open', 'adj_close', 'adj_ending',
       'adj_min', 'adj_max', 'Open', 'Close', 'log_adj_ending',
       'Low', 'High']

class SmaCross(Strategy):
    n1 = 10
    n2 = 30

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()


bt = Backtest(df, SmaCross, cash=10000, commission=0.015)

output = bt.run()
bt.plot()

In [ ]:
from pyalgotrade import strategy
from pyalgotrade.barfeed import quandlfeed


class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        super(MyStrategy, self).__init__(feed)
        self.__instrument = instrument

    def onBars(self, bars):
        bar = bars[self.__instrument]
        self.info(bar.getClose())

# Load the bar feed from the CSV file
feed = quandlfeed.Feed()
feed.addBarsFromCSV("orcl", "WIKI-ORCL-2000-quandl.csv")

# Evaluate the strategy with the feed's bars.
myStrategy = MyStrategy(feed, "orcl")
myStrategy.run()

In [16]:
df = pd.read_csv('./Data/qdl.csv')
len(df)

qdl.ApiConfig.api_key = getpass()
data = qdl.get("WIKI/AAPL", start_date="2007-01-01", end_date="2017-05-01")

data0 = data.copy()
data0['date_id'] = ((data0.index.date - data0.index.date.min())).astype('timedelta64[D]')
data0['date_id'] = data0['date_id'].dt.days + 1

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
x=np.arange(len(stocko))
plt.figure(num=None, figsize=(6, 4), dpi=150)
plt.scatter(highs, highs_price ,c="r")
xx=np.linspace(0,800,dtype=float)
plt.plot(xx,-0.5090909 * xx +1149.05858 , linestyle="-")
plt.plot(x,stocko,linestyle="-.")


········
